In [1]:
#r "BoSSSpad.dll"
#r "XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


init workflow


In [2]:
BoSSSshell.WorkflowMgm.Init("XESF_BowShock_OptProbStudy");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESF_BowShock_OptProbStudy'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESF_BowShock_OptProbStudy'.


{ Session Count = 6; Grid Count = 6; Path = \\dc3\userspace\sebastian\cluster\XESF_BowShock_OptProbStudy }

In [3]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-04-03 19:22:44Z

In [4]:
var sessions = database.Sessions;
sessions

#0: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 7:29:31 PM	2280ce24...
#1: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 7:20:53 PM	1af48180...
#2: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 7:20:47 PM	c86d97a4...
#3: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 7:26:39 PM	227da33c...
#4: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0*	4/3/2024 7:24:00 PM	6cc5fc7d...
#5: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0*	4/3/2024 7:22:44 PM	e92734cf...


In [5]:
MatrixAssembler aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

2280ce24-02cc-4c40-a745-462ac21dc71f

## Enthalpy error
enthalpy is computed as
$$h=\frac{(\rho E + p)}{\rho},~ p=(\gamma -1)(\rho E - \rho e_{kin}),~\rho e_{kin}=\frac{\rho }{2}(\vert u\vert^2 + \vert v\vert^2)$$

For an uniform inviscid flow the inflow enthalpy should remain constant throughout the flowfield 

In the case of the BowShock ($p_\text{in}=1,\rho_\text{in}=1$) presented, the v-velocity is zero and we have
$$ h_{in} = \rho E_{in}+1$$
We prescribe the inflow conditions by the Mach number $M_{in}=4$. From those the Energy is computed as 
$$ \rho E_{in} = (\rho e_{inner})_\text{in}+(\rho e_{kin})_\text{in}=\frac{p_\text{in}}{\gamma-1}+\frac{\rho_\text{in}}{2}u_\text{in}^2= \frac{1}{\gamma-1}+\frac{1}{2}u_{in}^2$$ 
where $u_{in} = M_{in}\sqrt{\gamma}$. In total we obtain:
$$ h_\text{in}=\rho E_{in}+1=\frac{\gamma}{\gamma-1}+\frac{1}{2}u_{in}^2$$


In [6]:
using BoSSS.Solution.CompressibleFlowCommon.MaterialProperty;
var enthalpy_end = (XDGField) si.Timesteps.Last().GetField("h");
var enthalpy_exact = (XDGField) enthalpy_end.Clone();
double gamma = IdealGas.Air.HeatCapacityRatio;
double Ms = 4;
double densityLeft = 1;
double pressureLeft = 1;
double velocityXLeft = Ms * Math.Sqrt(gamma * pressureLeft / densityLeft);

(gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft

14.7

In [7]:
var enthalpy_inflow = (gamma)/(gamma-1) + 0.5*velocityXLeft*velocityXLeft;
enthalpy_exact.Clear();
enthalpy_exact.GetSpeciesShadowField("L").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.GetSpeciesShadowField("R").ProjectField(1.0,t => enthalpy_inflow);
enthalpy_exact.Identification= "h_exact";
var EEN = enthalpy_exact.L2NormAllSpecies();

## Choose which runs to plot

In [14]:
var ses=sessions; // the one plotted in the publication
ses

#0: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 7:29:31 PM	2280ce24...
#1: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0	4/3/2024 7:20:53 PM	1af48180...
#2: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 7:20:47 PM	c86d97a4...
#3: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0	4/3/2024 7:26:39 PM	227da33c...
#4: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb1-iFlx1-wFLx0-bFlx0-Fphi0*	4/3/2024 7:24:00 PM	6cc5fc7d...
#5: XDGBowShock_TwoLs	XDGBS-p2-10x32-agg0.4-iPrb1-iFlx0-wFLx0-bFlx0-Fphi0*	4/3/2024 7:22:44 PM	e92734cf...


# A Helper Function to obtain different plot formats

In [15]:
using BoSSS.Solution.Tecplot;
using System.IO;
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Actual Enthalpy Error Plot


In [16]:
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    foreach(var timestep in si.Timesteps){
        var error = ((XDGField) timestep.GetField("h_err")).L2NormAllSpecies()/EEN;
        enthalpyerrors.Add(error);
    }
    allEE.Add(enthalpyerrors);
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Pick(enthalpyerrors.Count -1)).GetInternal();
    plot.AddDataGroup(si.Name,lts.TimeStepNumbers,enthalpyerrors, GetFormat(count));
    count++;
    enthalpyerrors.SaveToTextFile($"EnthalpyErrors_{si.Name}.txt");
}
plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 XDGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M511.2,57.1 L564.6,57.1 M45.6,185.2 L50.0,182.6 L54.4,181.3 L58.9,180.9 L63.3,179.9 L67.7,179.9
 L72.1,179.9 L76.5,179.8 L80.9,179.9 L85.4,179.9 L89.8,179.2 L94.2,178.8 L98.6,177.1 L103.0,179.4
 L107.5,179.9 L111.9,179.9 L116.3,179.9 L120.7,179.9 L125.1,179.9 L129.5,179.9 L134.0,179.9 L138.4,179.9
 L142.8,180.0 L147.2,179.0 L151.6,179.7 L156.0,181.4 L160.5,182.8 L164.9,182.9 L169.3,183.0 L173.7,182.1
 L178.1,184.3 L182.6,184.7 L187.0,184.1 L191.4,184.3 L195.8,183.6 L200.2,182.6 L204.6,182.6 L209.1,182.6
 L213.5,182.5 L217.9,182.4 L222.3,182.4 L226.7,182.4 L231.2,182.4 L235.6,157.9 L240.0,161.5 L244.4,162.6
 L248.8,162.2 L253.2,162.1 L257.7,162.1 L262.1,162.1 L266.5,162.1 L270.9,162.1 L275.3,162.1 L279.8,162.1
 L284.2,162.1 L288.6,162.1 L293.0,162.1 L297.4,162.1 L301.8,162.1 L306.3,162.1 L310.7,162.1 L315.1,162.1
 L319.5,162.1 L323.9,162.1 L328.4,162.1 L332.8,162.1 L337.2,162.1 L341.6,162.1 L346.0,162.1 L350.4,162.1
 L354.9,162.1 L359.3,162.1 L363.7,162.1 L368.1,162.1 L372.5,162.1 L376.9,162.1 L381.4,162.1 L385.8,162.1
 L390.2,162.1 L394.6,162.1 L399.0,162.1 L403.5,162.1 L407.9,162.1 L412.3,162.1 L416.7,162.1 L421.1,162.1
 L425.5,162.1 L430.0,162.1 L434.4,162.1 L438.8,162.1 L443.2,162.1 L447.6,162.1 L452.1,162.1 L456.5,162.1
 L460.9,162.1 L465.3,162.1 L469.7,162.1 L474.1,162.1 L478.6,162.1 L483.0,162.1 L487.4,162.1 L491.8,162.1
 L496.2,162.1 L500.7,162.1 L505.1,162.1 L509.5,162.1 L513.9,162.1 L518.3,162.1 L522.7,162.1 L527.2,162.1
 L531.6,162.1 L536.0,162.1 L540.4,162.1 L544.8,162.1 L549.3,162.1 L553.7,162.1 L558.1,162.1 L562.5,162.1
 L566.9,162.1 L571.3,162.1 L575.8,162.1 L580.2,162.1 L584.6,162.1 L589.0,162.1 L593.4,162.1 L597.8,162.1
 L602.3,162.1 L606.7,162.1 L611.1,162.1 L615.5,162.1 L619.9,162.1 L624.4,162.1 L628.8,162.1 L633.2,162.1
 L637.6,162.1 L642.0,162.1 L646.4,162.1 L650.9,162.1 L655.3,162.1 L659.7,162.1 L664.1,162.1 L668.5,162.1
 L673.0,162.1 L677.4,162.1 L681.8,162.1 L686.2,162.1 L690.6,162.1 L695.0,162.1 L699.5,162.1 L703.9,162.1
 L708.3,162.1 L712.7,162.1 L717.1,162.1 L721.6,162.1 L726.0,162.1 L730.4,162.1 L734.8,162.1 L739.2,162.1
 L743.6,162.1 L748.1,162.1 L752.5,162.1 L756.9,162.1 L761.3,162.1 L765.7,162.1 L770.2,162.1 L774.6,162.1
 L779.0,162.1 L783.4,162.1 L787.8,162.1 L792.2,162.1 L796.7,162.1 L801.1,162.1 L805.5,162.1 L809.9,162.1
 L814.3,162.1 L818.7,162.1 L823.2,162.1 L827.6,162.1 L832.0,162.1 L836.4,162.1 L840.8,162.1 L845.3,162.1
 L849.7,162.1 L854.1,162.1 L858.5,162.1 L862.9,162.1 L867.3,162.1 L871.8,162.1 L876.2,162.1 L880.6,162.1
 L885.0,162.1 L889.4,162.1 L893.9,162.1 L898.3,162.1 L902.7,162.1 L907.1,162.1 L911.5,162.1 L915.9,162.1
 L920.4,162.1 L924.8,162.1 L929.2,162.1 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M511.2,81.1 L564.6,81.1 M45.6,185.2 L50.0,163.8 L54.4,168.2

## Residual History

In [17]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var texplot = new Tecplot(enthalpy_exact.GridDat, 2);
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in ses){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    lts.ResHistory.SaveToTextFile($"ResHistory_{si.Name}.txt");
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    lts.EnResHistory.SaveToTextFile($"EnResHistory_{si.Name}.txt");
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 10 4 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - XDGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 ||r|| - XDGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M422.4,57.1 L475.8,57.1 M53.9,79.6 L58.3,80.4 L62.7,81.5 L67.1,82.3 L71.4,82.7 L75.8,82.8
 L80.2,83.0 L84.6,83.1 L89.0,83.1 L93.4,83.1 L97.7,83.2 L102.1,83.9 L106.5,83.6 L110.9,84.5
 L115.3,84.7 L119.7,84.7 L124.1,84.7 L128.4,84.7 L132.8,84.7 L137.2,84.7 L141.6,84.7 L146.0,84.7
 L150.4,84.1 L154.8,84.1 L159.1,84.0 L163.5,87.5 L167.9,87.8 L172.3,88.0 L176.7,87.8 L181.1,88.8
 L185.4,86.9 L189.8,86.8 L194.2,87.9 L198.6,87.4 L203.0,87.5 L207.4,87.5 L211.8,85.0 L216.1,85.2
 L220.5,87.0 L224.9,87.1 L229.3,87.2 L233.7,87.2 L238.1,87.2 L242.4,72.0 L246.8,72.1 L251.2,72.1
 L255.6,72.2 L260.0,72.2 L264.4,72.2 L268.8,72.2 L273.1,72.2 L277.5,72.2 L281.9,72.2 L286.3,72.2
 L290.7,72.2 L295.1,72.2 L299.4,72.2 L303.8,72.2 L308.2,72.2 L312.6,72.2 L317.0,72.2 L321.4,72.2
 L325.8,72.2 L330.1,72.2 L334.5,72.2 L338.9,72.2 L343.3,72.2 L347.7,72.2 L352.1,72.2 L356.5,72.2
 L360.8,72.2 L365.2,72.2 L369.6,72.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResXDGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 EnRes X DGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M422.4,81.1 L475.8,81.1 M53.9,77.0 L58.3,75.7 L62.7,76.2 L67.1,76.8 L71.4,77.5 L75.8,77.6
 L80.2,77.7 L84.6,77.8 L89.0,77.8 L93.4,77.8 L97.7,78.4 L102.1,79.4 L106.5,79.8 L110.9,80.9
 L115.3,81.1 L119.7,81.1 L124.1,81.1 L128.4,81.1 L132.8,81.1 L137.2,81.1 L141.6,81.1 L146.0,81.1
 L150.4,81.0 L154.8,81.4 L159.1,83.4 L163.5,83.7 L167.9,84.3 L172.3,84.4 L176.7,84.6 L181.1,85.9
 L185.4,87.0 L189.8,87.1 L194.2,87.5 L198.6,87.2 L203.0,88.2 L207.4,89.1 L211.8,85.2 L216.1,85.4
 L220.5,87.1 L224.9,86.9 L229.3,87.0 L233.7,87.0 L238.1,87.0 L242.4,70.5 L246.8,70.6 L251.2,70.6
 L255.6,70.6 L260.0,70.6 L264.4,70.6 L268.8,70.6 L273.1,70.6 L277.5,70.6 L281.9,70.6 L286.3,70.6
 L290.7,70.6 L295.1,70.6 L299.4,70.6 L303.8,70.6 L308.2,70.6 L312.6,70.6 L317.0,70.6 L321.4,70.6
 L325.8,70.6 L330.1,70.6 L334.5,70.6 L338.9,70.6 L343.3,70.6 L347.7,70.6 L352.1,70.6 L356.5,70.6
 L360.8,70.6 L365.2,70.6 L369.6,70.6 L374.0,65.5 L378.4,65.5 L382.8,65.5 L387.1,65.5 L391.5,65.5
 L395.9,65.5 L400.3,65.5 L404.7,65.5 L409.1,65.5 L413.5,65.5 L417.8,65.5 L422.2,65.5 L426.6,65.5
 L431.0,65.5 L435.4,65.5 L439.8,65.5 L444.1,65.5 L448.5,65.5 L452.9,65.5 L457.3,65.5 L461.7,65.5
 L466.1,65.5 L470.5,65.5 L474.8,65.5 L479.2,65.5 L483.6,65.5 L488.0,65.5 L492.4,65.5 L496.8,65.5
 L501.1,65.5 L505.5,65.5 L509.9,65.5 L514.3,65.5 L518.7,65.5 L523.1,65.5 L527.5,65.5 L531.8,65.5
 L536.2,65.5 L540.6,65.5 L545.0,65.5 L549.4,65.5 L553.8,65.5 L558.2,65.5 L562.5,65.5 L566.9,65.5
 L571.3,65.5 L575.7,65.5 L580.1,65.5 L584.5,65.5 L588.8,65.5 L593.2,65.5 L597.6,65.5 L602.0,65.5
 L606.4,65.5 L610.8,65.5 L615.2,65.5 L619.5,65.5 L623.9,65.5 L628.3,65.5 L632.7,65.5 L637.1,65.5
 L641.5,65.5 L645.8,65.5 L650.2,65.5 L654.6,65.5 L659.0,65.5 L663.4,65.5 L667.8,65.5 L672.2,65.5
 L676.5,65.5 L680.9,65.5 L685.3,65.5 L689.7,65.5 L694.1,65.5 L698.5,65.5 L702.9,65.5 L707.2,65.5
 L711.6,65.5 L716.0,65.5 L720.4,65.5 L724.8,65.5 L729.2,65.5 L733.5,65.5 L737.9,65.5 L742.3,65.5
 L746.7,65.

### Plot ShadowFields

In [ ]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
foreach(var si in ses){
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    var flds=lts.GetShadowFields();
    var tp= new Tecplot(flds.Last().GridDat,2);
    tp.PlotFields(si.Name,0,flds);
}

rm XDGBS-p2-10x32-agg0.4-iPrb0-iFlx0-wFLx0-bFlx0-Fphi0.plt XDGBS-p2-10x32-agg0.4-iPrb0-iFlx1-wFLx0-bFlx0-Fphi0.plt XDGBS-p2-10x32-agg0.4-iPrb2-iFlx0-wFLx0-bFlx0-Fphi0.plt XDGBS-p2-10x32-agg0.4-iPrb2-iFlx1-wFLx0-bFlx0-Fphi0.plt;
